In [123]:
import pandas as pd
import os
import numpy as np

In [124]:
data1 = pd.read_csv('datasets_11796_16250_ionosphere_data_kaggle.csv')
data1.head()


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [125]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature1   351 non-null    int64  
 1   feature2   351 non-null    int64  
 2   feature3   351 non-null    float64
 3   feature4   351 non-null    float64
 4   feature5   351 non-null    float64
 5   feature6   351 non-null    float64
 6   feature7   351 non-null    float64
 7   feature8   351 non-null    float64
 8   feature9   351 non-null    float64
 9   feature10  351 non-null    float64
 10  feature11  351 non-null    float64
 11  feature12  351 non-null    float64
 12  feature13  351 non-null    float64
 13  feature14  351 non-null    float64
 14  feature15  351 non-null    float64
 15  feature16  351 non-null    float64
 16  feature17  351 non-null    float64
 17  feature18  351 non-null    float64
 18  feature19  351 non-null    float64
 19  feature20  351 non-null    float64
 20  feature21 

In [126]:
data1["label"].value_counts()

g    225
b    126
Name: label, dtype: int64

In [127]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["label"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [128]:
 strat_test_set["label"].value_counts()

g    56
b    32
Name: label, dtype: int64

In [129]:
strat_train_set["label"].value_counts()

g    169
b     94
Name: label, dtype: int64

In [130]:
train_set = strat_train_set.drop("label", axis=1) # drop labels for training set
train_labels = strat_train_set["label"].copy()
test_set = strat_test_set.drop("label", axis=1) # drop labels for testing set
test_labels = strat_test_set["label"].copy()

In [131]:
train_labels

79     b
312    g
46     g
218    b
344    g
      ..
10     g
217    g
71     b
134    b
193    g
Name: label, Length: 263, dtype: object

In [132]:
test_labels

181    g
122    b
186    b
285    g
128    b
      ..
29     b
238    b
177    g
144    b
139    g
Name: label, Length: 88, dtype: object

In [133]:
!pip install fuzzy-c-means
from fcmeans import FCM
tr_set=train_set.T
fcm = FCM(n_clusters=20,random_state=42)
fcm.fit(tr_set.to_numpy())
tr_fin  = fcm.centers.T

In [134]:
te_set=test_set.T
fcm = FCM(n_clusters=20, random_state=42)
fcm.fit(te_set.to_numpy())
te_fin  = fcm.centers.T

In [135]:
tr_fin.shape

(263, 20)

In [136]:
train_labels.shape

(263,)

In [137]:
from sklearn.metrics import matthews_corrcoef
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
parameters = {"loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
              "alpha" : [0.0001, 0.001, 0.01, 0.1],
              "penalty" : ["l2", "l1", "elasticnet", "none"],
              "tol": [1.e-3]
}


clf2 = SGDClassifier(early_stopping= True, max_iter=10000, random_state=42)
clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)
res1=clf.fit(tr_fin, train_labels)
res1.best_estimator_

SGDClassifier(alpha=0.01, average=False, class_weight=None, early_stopping=True,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='squared_hinge', max_iter=10000,
              n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [138]:
res1.best_params_

{'alpha': 0.01, 'loss': 'squared_hinge', 'penalty': 'none', 'tol': 0.001}

In [139]:
res1.best_score_

0.7164624183006536

In [140]:
y_test_pred=res1.best_estimator_.predict(te_fin)


In [141]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

48.86363636363637

In [142]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

51.13636363636363

In [143]:
matthews_corrcoef(test_labels, y_test_pred)*100

9.31331260378521

In [144]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label= 'b')*100

39.34426229508197

In [145]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label= 'b')*100

75.0

In [146]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred,  pos_label= 'b')*100

51.61290322580645